In [8]:
import jax
import genjax

key = jax.random.PRNGKey(314159)
console = genjax.pretty()

In [13]:
@genjax.gen
def model(t):
    x = genjax.tfp_normal(0.0, 100.0) @ "x"
    v = jax.lax.cond(t == 0, lambda: x, lambda: 0.0)
    y = genjax.tfp_normal(v, 1.0) @ "y"

In [14]:
tr = model.simulate(key, (1,))

In [18]:
@genjax.gen
def forward_proposal(trace, obs_y):
    y = obs_y["y"]
    new_x = genjax.tfp_normal(y, 1.0) @ "new_x"
    return (genjax.choice_map({"x": new_x}), genjax.choice_map({"x": trace["x"]}))


@genjax.gen
def backward_proposal(trace, obs_y):
    x = genjax.tfp_normal(0.0, 1.0) @ "x"
    return (genjax.choice_map({"x": x}), genjax.choice_map({"new_x": x}))

In [ ]:
def smcp3_update(key, trace, new_obs_y):
    key, sub_key = jax.random.split(key)
    (choices, forward_weight, (chm, aux)) = forward_proposal.propose(
        sub_key, (trace, new_obs_y)
    )
    key, sub_key = jax.random.split(key)
    (_, w, new_trace, _) = model.update(
        sub_key, trace, chm, genjax.tree_diff_unknown_change((0,))
    )
    (retval, backward_weight) = backward_proposal.assess(
        sub_key, aux, (new_trace, new_obs_y)
    )
    tr_ker_fwd = trace_kernel(forward_proposal)
    J = tr_key_fwd.jacfwd_retval(key, choices, trace, (new_obs_y,))
    sign, val = jnp.linalg.slogdet(J)